# Notebook that generates my README

In [ ]:
import json
from collections import Counter
from datetime import datetime

import jinja2
import matplotlib
import matplotlib.pyplot as plt
import mplcyberpunk

import generate_data as gd

plt.style.use("cyberpunk")

# matplotlib.rcParams.update({"font.family": ["Humor Sans"]})  # avoids findfont warnings

In [ ]:
_ = await gd.generate_data()

In [ ]:
with open("data/repos.json", "r") as f:
    repos = json.load(f)

with open("data/most_committed.json", "r") as f:
    most_committed = json.load(f)

with open("data/stargazers.json", "r") as f:
    stargazers = json.load(f)
    for info in stargazers:
        info["dates"] = [datetime.fromisoformat(date) for date in info["dates"]]

with open("data/all_commit_dates.json", "r") as f:
    all_commit_dates = [datetime.fromisoformat(date) for date in json.load(f)]

## Number of stars ⭐️

In [ ]:
most_stars_strs = [
    f"{i+1}. [{repo['full_name']}](https://github.com/{repo['full_name']}/), {repo['stargazers_count']} ⭐️s"
    for i, repo in enumerate(stargazers[:20])
]

most_stars_strs_summary, most_stars_strs_rest = gd.split(most_stars_strs, 5)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.set_xlabel("Year")
ax.set_ylabel("Cumulative number of stars")
ax.set_title("Total number of stars over time")

for star_info in stargazers:
    dates = star_info["dates"]
    n_stars = list(range(1, len(dates) + 1))
    ax.plot(dates, n_stars, label=star_info["full_name"])
mplcyberpunk.add_glow_effects()
plt.savefig("stars_over_time_per_repo.png")

In [ ]:
all_dates = sorted([date for star_info in stargazers for date in star_info["dates"]])
n_stars = list(range(1, len(all_dates) + 1))

fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(all_dates, n_stars)
ax.set_xlabel("Year")
ax.set_ylabel("Cumulative number of stars")
ax.set_title("Total number of stars over time")
mplcyberpunk.add_glow_effects()
plt.savefig("stars_over_time.png")

In [ ]:
# gd.to_plotly_json()

# import plotly.graph_objs as go
# from plotly.subplots import make_subplots

# with open("data/traces_data.json", "r") as infile:
#     traces_data = json.load(infile)
# fig = make_subplots()
# for trace_data in traces_data:
#     trace = go.Scatter(**trace_data)
#     fig.add_trace(trace)
# fig.update_layout(
#     title="Cumulative number of stars over time",
#     xaxis_title="Date",
#     yaxis_title="Number of stars",
# )
# fig.show()


## Number of commits 

In [ ]:
most_committed_strs = [
    f"{i+1}. [{full_name}](https://github.com/{full_name}/), {n_commits} commits :octocat:"
    for i, (full_name, n_commits) in enumerate(most_committed[:20])
]

most_committed_strs_summary, most_committed_strs_rest = gd.split(most_committed_strs, 5)
most_committed_strs

## Commit stats

In [ ]:
day_hist, hour_hist = gd.generate_day_hour_histograms(all_commit_dates)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))
bars = ax.bar(*zip(*day_hist), color=[f"C{i}" for i in range(7)])
ax.set_xlabel("Day of the week")
ax.set_ylabel("Total number of commits")
ax.set_title("Commits by day")
mplcyberpunk.add_bar_gradient(bars=bars)
plt.savefig("commits_per_weekday.png")

In [ ]:
hour_hist = [
    (f"{i:02d}", n)
    for i, n in sorted(Counter([d.hour for d in all_commit_dates]).items())
]

fig, ax = plt.subplots(figsize=(9, 5))
bars = ax.bar(*zip(*hour_hist))
ax.set_xlabel("Hour of the day")
ax.set_ylabel("number of commits")
ax.set_title("Commits by hour")
mplcyberpunk.add_bar_gradient(bars=bars)
plt.savefig("commits_per_hour.png")

## Render template

In [ ]:
with open("README.jinja2", "r") as f:
  template = f.read()
txt = jinja2.Template(template).render(
    most_stars_strs_summary=most_stars_strs_summary,
    most_stars_strs_rest=most_stars_strs_rest,
    most_committed_strs_summary=most_committed_strs_summary,
    most_committed_strs_rest=most_committed_strs_rest,
    now=str(datetime.now()),
)
with open("README.md", "w") as f:
    f.write(txt)
print(txt)